# TO-DO
* Figure out what each variables are
* Figure out the relevant brain regions ==> in the ccfNames variable
* Find out what is the input and output for the network
* Setup the network in advance ==> while figuring out other stuff
* Setup function for extracting all probes from all rats
    * All **eight** probes 
    * All **three** rats
* Training phase
    * k-fold cross validation ???
    * ROC curve ???

In [40]:
import scipy.io
import numpy as np
import pip

# pip.main(['install'] + ['tensorflow'])
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Decode Mouse Movements
### Using recurrent neural network (RNN), we are going to decode mouse movements from neural activity in various parts of the mouse brain.


Dataset is from https://figshare.com/articles/Eight-probe_Neuropixels_recordings_during_spontaneous_behaviors/7739750, with the data as follows,
* **probeLocations** ==> Information about each probe's location
    * ccfCoords ==> Location of each site on the probe in the microns in the Allen CCF framework
* **probeBorders** ==> 

The "spks" each is a structure of length 8, where each entry is a different probe (these probes were recorded simultaneously). It contains the spike times and the cluster identity of each spike.
* **spks(k).st** = spike times in seconds
* **spks(k).clu** = cluster identity of each spike in st (which neuron does spike belong to)
* **spks(k).Wheights** = height of each cluster on the probe

In our notebook, we specify them as **st**, **clu** and **Wh**, respectively


### NOTE: These data are for MATLAB, and are .mat file type




In [47]:
# From the following link https://github.com/MouseLand/stringer-pachitariu-et-al-2018a

from scipy import io

probeLoc = io.loadmat('probeLocations.mat')
probeBorders = io.loadmat('probeBorders.mat', squeeze_me=True)

mouse_names = ['Krebs','Waksman','Robbins']
# start of spontaneous activity in each mouse (in seconds)
tstart = [3811, 3633, 3323];

## Function for extracting each mouse's parameters
def extract_data(k,imouse):
## k is the number of probe (0 to 7)
## imouse is the list containing mouse names (0 to 2)
    if (k in [i for i in range(8)]) and (imouse in [i for i in range(2)]):
# imouse = 0

        spks = io.loadmat('Spikes/spks%s_Feb18.mat'%mouse_names[imouse], squeeze_me=True)
        faces = io.loadmat('Faces/%s_face_proc.mat'%mouse_names[imouse], squeeze_me=True)

        # probe k
        # k = 0
        # spike times (in seconds)
        st = spks['spks'][k][0]   # (606058,)
        # clusters
        clu = spks['spks'][k][1]  # (606058,)
        # cluster heights (in microns) ==> Height of each cluster on the probe
        # (see siteCoords to convert to site location)
        Wh = spks['spks'][k][2]    # (127,)

        # processed faces
        motSVD = faces['motionSVD']
        video_timestamps = faces['times']

        # where is the probe in the brain (consolidated labels)
        # borders are in microns
        # use Wh to determine which clusters are in which brain region
        borders = probeBorders['probeBorders'][imouse]['borders'][k]
        for j in range(len(borders)):
           b = borders[j]
           print('upper border %d, lower border %d, area %s'%(b[0],b[1],b[2]))
           wneurons = np.logical_and(Wh>=b[1], Wh<b[0])
           nn = wneurons.sum()
           print('%d neurons in %s'%(nn,b[-1]))

        # where is the probe in the brain (in microns) ==> (384,3)
        ccfCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfCoords']
        # name of area in Allen ontology by site on electrode ==> (384,1)
        ccfNames = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfOntology']
        # coordinates of each site on the electrode ==> (384,2)
        siteCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['siteCoords']
    elif (k in [i for i in range(8)]) and (imouse not in [i for i in range(2)]):
        raise Exception('imouse should be the integer from 0 to 2 only.')
    elif (k not in [i for i in range(8)]) and (imouse in [i for i in range(2)]):
        raise Exception('i should be the integer from 0 to 7 only.')
    else:
        raise Exception('Both of your function inputs are not eligible as i needs to be integer from 0 to 7 and imouse needs to be integer from 0 to 2 only.')
    
    return st,clu,Wh,motSVD,video_timestamps,ccfCoords,ccfNames,siteCoords


## Relevant Anatomy
From ccfNames variable, we have found the following brain regions that are relevant to the data
* Bed nucleus of the stria terminalis (BST) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4011829/
    * Its connectivity suggest it being a **relay center** within neurocircuits coordinating the activity of **autonomic, neuroendocrine, and somatic motor systems** into fully organized physiological functions and behaviors
    * Info flows into BST from exteroceptive or interoceptive sources through all CNS system levels
    * Lesions or pharmacological manipulations ==> Physiology of fear, food intake, social behaviours, pain, and goal-directed behaviours and the associated pathophysiological states such as anxiety, anorexia, and addiction

* Lateral Septal Ventral (LSv)
    

In [49]:
## Playground
# See what is inside the variables inside probeLoc
# display(ccfCoords)
# display(ccfCoords.shape)
# display(ccfNames)
# display(ccfNames.shape)
# display(siteCoords)
# display(siteCoords.shape)
# extract_data(0,0)
# See what is inside the variables inside spks
# display(st)
# display(st.shape)
# display(clu)
# display(clu.shape)
# display(Wh)
# display(Wh.shape)
# display(len(spks['spks'])   # This is 8, for 8 probes recording simultaneously
# display(spks)
# display(mouse_names[0])
# display([i for i in range(8)])
print('Lol my name is %s'%mouse_names[0])

upper border 4000, lower border 1100, area FrMoCtx
5 neurons in FrMoCtx
upper border 1100, lower border 0, area LS
122 neurons in LS
Lol my name is Krebs


## Recurrent Neural Network (RNN)
RNN processes sequences, one element at a time, and it has memory, so it is a causal system(?). With this memory, it can use the entire context to take into account when making a prediction, which is what we process sequences, considering the whole sentence before forming a response.


Source: https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470

In [ ]:
### Recurrent Neural Network ###


# Appendix
## Appendix A: Relevant Papers

* Spontaneous Behaviors Drive Multidimensional, Brain-wide Activity ==> All the spike-sorting electrophysiology and the corresponding behavioral traces are referred to this dataset
    * https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6525101/